# Model training

We decide to train the model with xgboost. We run through grid search for hyperparameters with hyperopt_xgb notebook. Since it is too time consuming, we stop it halfway, and tune them manually on top of the given suboptimized hyperparameters computed.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
from multiprocessing import Pool
import xgboost as xgb
from itertools import product
import pickle
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [ ]:
test = pd.read_csv('test.csv')
X_train = pd.read_csv('X_train.csv')
X_cv = pd.read_csv('X_cv.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
params = {
        'eta': 0.08, #best 0.08
        'max_depth': 7,
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'seed': 3,
        'gamma':1,
        'silent': True
    }

In [4]:
cols = [c for c in X_train.columns if c not in ['date_block_num', 'item_cnt_day',]]

x1 = X_train[cols]
y1 = X_train['item_cnt_day']
x2 = X_cv[cols]
y2 = X_cv['item_cnt_day']
watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
model = xgb.train(params, xgb.DMatrix(x1, y1), 3500,  watchlist, maximize=False, verbose_eval=50, early_stopping_rounds=50)

[0]	train-rmse:3.39355	valid-rmse:2.61947
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.82179	valid-rmse:1.87885
[100]	train-rmse:1.76671	valid-rmse:1.8706
[150]	train-rmse:1.71617	valid-rmse:1.86389
[200]	train-rmse:1.67721	valid-rmse:1.85492
[250]	train-rmse:1.63726	valid-rmse:1.8364
[300]	train-rmse:1.61528	valid-rmse:1.83118
Stopping. Best iteration:
[298]	train-rmse:1.61571	valid-rmse:1.83075



In [5]:
pickle.dump(model, open("xgb.pickle.dat", "wb"))
model = pickle.load(open("pima.pickle.dat", "rb"))

pred = model.predict(xgb.DMatrix(X_test[cols]), ntree_limit=model.best_ntree_limit)

test['item_cnt_month'] = pred.clip(0,20)
test.drop(['shop_id', 'item_id'], axis=1, inplace=True)
test.to_csv('submission.csv', index=False)

# Result

[0]	train-rmse:1.40592	valid-rmse:1.37868
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.03644	valid-rmse:1.14389
[100]	train-rmse:1.00372	valid-rmse:1.13304
[150]	train-rmse:0.987922	valid-rmse:1.12918
[200]	train-rmse:0.972436	valid-rmse:1.12584
[250]	train-rmse:0.960087	valid-rmse:1.12552
[300]	train-rmse:0.949489	valid-rmse:1.12458
[350]	train-rmse:0.940702	valid-rmse:1.12259
[400]	train-rmse:0.932332	valid-rmse:1.12257
Stopping. Best iteration:
[352]	train-rmse:0.940543	valid-rmse:1.12231

Due to the random seed, our score is roughly about 0.95 in public leaderboard and private leaderboard.
